In [0]:
import keras
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
from google.colab.patches import cv2_imshow
from numpy import save,load
import keras 
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import random
import tensorflow as tf
import cv2

from keras import models
from keras import layers 
from keras.applications import ResNet50
import matplotlib.pyplot as plt
carpet='Entrance26'

start = keras.Input(shape=(32,32,3))
Conv1 =layers.Conv2D(64,(3,3),padding='same',activation='relu')(start)
Conv2 =layers.Conv2D(64,(3,3),padding='same',activation='relu')(Conv1)
Pool1 =layers.MaxPooling2D(pool_size=(2,2))(Conv2)

Conv3 =layers.Conv2D(128,(3,3),padding='same',activation='relu')(Pool1)
Conv4 =layers.Conv2D(128,(3,3),padding='same',activation='relu')(Conv3)
Pool2 =layers.MaxPooling2D(pool_size=(2,2))(Conv4)

Conv5 =layers.Conv2D(256,(3,3),padding='same',activation='relu')(Pool2)
Conv6 =layers.Conv2D(256,(3,3),padding='same',activation='relu')(Conv5)
Pool3 =layers.MaxPooling2D(pool_size=(2,2))(Conv6)

Flat =layers.Flatten()(Pool3)
Den1 =layers.Dense(4*4*256,activation='relu')(Flat)
Den2  =layers.Dense(256,activation='relu')(Den1)
Den3  =layers.Dense(1,activation='sigmoid')(Den2)

model=keras.Model(start,Den3)

safe_acc=[]
safe_val_acc=[]
safe_loss=[]
safe_val_loss=[]

model.load_weights('/content/drive/My Drive/CNN2/Eyes/Trained Models/Entrance26/Checkpoints/Checkpoint_weight_49_0.06_0.98_0.00_0.95_.h5')
X=[]

for filename in os.listdir('../TryOut/'): #for each file in that directory
    print(filename)
    photo=img_to_array(load_img('../TryOut/'+filename))    
    X.append(photo)

cropped=[]
sq=16
detector=MTCNN()

temp=[]
for sample in range(len(X)):
    print(sample)
    temp_photo=np.array(X[sample])
    temp_faces=detector.detect_faces(temp_photo)
    faces=len(temp_faces)
    for i in range(faces):
      x=temp_faces[i]['box'][0]
      y=temp_faces[i]['box'][1]
      w=temp_faces[i]['box'][2]
      h=temp_faces[i]['box'][3]
      croppedFace=temp_photo[y:y+h,x:x+w,:]
      croppedFace=cv2.resize(croppedFace,(100,100))
      temp=detector.detect_faces(croppedFace)
      if(len(temp)!=0):
        x1,y1=temp[0]['keypoints']['left_eye']
        croppedL=croppedFace[y1-sq:y1+sq,x1-sq:x1+sq,:]
        x,y=temp[0]['keypoints']['right_eye']
        croppedR=croppedFace[y-sq:y+sq,x-sq:x+sq,:]
        croppedL=croppedL.reshape((1,32,32,3))
        croppedR=croppedR.reshape((1,32,32,3))
        if(model.predict(croppedL)==1):
          if(model.predict(croppedR)==1):
            print('Closed')
          else:
            print('Open')
        else:
          print('Open')
        croppedFace=cv2.rectangle(croppedFace,(x-sq,y-sq),(x+sq,y+sq),(255,0,0),(1))
        croppedFace=cv2.rectangle(croppedFace,(x1-sq,y1-sq),(x1+sq,y1+sq),(255,0,0),(1))
        cv2_imshow(croppedFace)